In [ ]:
from os.path import join, exists
from dataclasses import asdict

import numpy

from line_searches import make_line_search, ArmijoLineSearch, WolfeLineSearch, BrentLineSearch
from optimizers import make_optimizer
from oracles import make_oracle
from experiment import experiment_runner, experiment_graphic_builder
from config import Config

%load_ext autoreload
%autoreload 2

# Логистическая регрессия

Задача в матричном виде:
$$
F_{train}(w) = -\frac{1}{n} \Big( y^T \cdot \log (\sigma(Xw)) + (1 - y^T) \cdot \log (1 - \sigma(Xw)) \Big)
$$

Для неё можно вычислить значения градиента:
$$
\nabla_W F = \frac{1}{n} X^T (\sigma(Xw) - y)
$$
Для вычисления гессиана определим вспомогательную диагональную матрицу $D$:
$$
D = \text{diag}\Big( \frac{1}{n} \sigma(Xw) (1 - \sigma(Xw)) \Big)
$$
Тогда
$$
H_F(w) = X^T D X
$$

Рассмотрим гессиан и некоторый вектор $a$. Воспользуемся диагональностью $D$, а также что на её диагонали положительный элементы (множество значений логит-функции $-$ $[0; 1]$):
$$
a^T H_F(w) a = a^T X^T D X a = a^T X^T \sqrt{D} \sqrt{D} X a = ||\sqrt{D} X a|| \ge 0
$$
Получили, что гессиан логистической регрессии положительно определён. Тогда исходная задача выпуклая. Поэтому результаты оптимизации функции не зависят от выбранной стартовой точки.

# Инициализация

Вспомогательный блок для инициализации оракулов и методов одномерной оптимизации

In [ ]:
config = Config()
print('\n'.join((f"{key}: {value}" for key, value in asdict(config).items())))

In [ ]:
line_search_methods = ["golden", "brent", "armijo", "wolfe", "nesterov"]
line_searches = [make_line_search(name, config) for name in line_search_methods]

In [ ]:
a1a_data_path = join(Config.data_folder, "a1a.txt")
if not exists(a1a_data_path):
    raise RuntimeError(f"Can't find a1a dataset in {a1a_data_path}")
a1a_oracle = make_oracle(a1a_data_path)
a1a_start_point = numpy.zeros(a1a_oracle.n_features)
a1a_true_minimum = a1a_oracle.get_true_minimum(Config.tol, Config.max_iter)

In [ ]:
bc_data_path = join(Config.data_folder, "breast-cancer_scale.txt")
if not exists(bc_data_path):
    raise RuntimeError(f"Can't find a1a dataset in {bc_data_path}")
bc_oracle = make_oracle(bc_data_path)
bc_start_point = numpy.zeros(bc_oracle.n_features)
bc_true_minimum = bc_oracle.get_true_minimum(Config.tol, Config.max_iter)

In [ ]:
random_data_path = join(Config.data_folder, "random.tsv")
if not exists(random_data_path):
    print(f"Can't found random data on {random_data_path}, will generate new")
    parameters = numpy.random.uniform(low=-1, high=1, size=(51,))
    x = numpy.random.rand(1000, 1)
    value = (x * parameters[:-1]).sum(axis=1) + parameters[-1]
    y = (value >= 0).astype(numpy.short).reshape(-1, 1)
    numpy.savetxt(random_data_path, numpy.append(y, x, axis=1), delimiter='\t', fmt=["%d", "%8e"])
    print(f"Generate 1000 samples. Class balance: {y.sum()}/{1000-y.sum()}")
random_oracle = make_oracle(random_data_path, data_format="tsv")
random_start_point = numpy.zeros(random_oracle.n_features)
random_true_minimum = random_oracle.get_true_minimum(Config.tol, Config.max_iter)

# Оптимизация методом градиентного спуска

In [ ]:
grad_descent_optimizer = make_optimizer("gradient descent", config)

## a1a dataset

In [ ]:
%%time
a1a_optim_res = experiment_runner(a1a_oracle, line_searches, grad_descent_optimizer, a1a_start_point)

In [ ]:
a1a_figure = experiment_graphic_builder(
    a1a_optim_res,
    "Compare line search methods on a1a dataset using gradient descent optimizer",
    a1a_true_minimum,
    smooth=True
)
a1a_figure.show()

## Breast cancer (scaled)

In [ ]:
%%time
bc_optim_res = experiment_runner(bc_oracle, line_searches, grad_descent_optimizer, bc_start_point)

In [ ]:
bc_figure = experiment_graphic_builder(
    bc_optim_res,
    "Compare line search methods on breast cancer (scaled) dataset using gradient descent optimizer",
    bc_true_minimum,
    smooth=True
)
bc_figure.show()

## Random dataset

In [ ]:
%%time
random_optim_res = experiment_runner(random_oracle, line_searches, grad_descent_optimizer, random_start_point)

In [ ]:
random_figure = experiment_graphic_builder(
    random_optim_res,
    "Compare line search methods on random dataset using gradient descent optimizer",
    random_true_minimum,
    smooth=True
)
random_figure.show()

## Выбор константы для условия Армихо

In [ ]:
armijo_line_searches = []
for armijo_c in [1e-4, 1e-2, 0.1, 0.5, 0.9]:
    armijo_config = Config()
    armijo_config.armijo_c = armijo_c
    line_search = ArmijoLineSearch(armijo_config)
    line_search.name = f"{line_search.name} (c={armijo_c})"
    armijo_line_searches.append(line_search)

In [ ]:
armijo_optim_res = experiment_runner(
    a1a_oracle, armijo_line_searches, grad_descent_optimizer, a1a_start_point
)

In [ ]:
armijo_figure = experiment_graphic_builder(
    armijo_optim_res,
    "Compare different parameters for Armijo condition",
    a1a_true_minimum,
    smooth=True
)
armijo_figure.show()

## Выбор константы для сильно условия Вульфа

In [ ]:
wolfe_line_searches = []
for wolfe_c in [1e-4, 1e-2, 0.1, 0.5, 0.9]:
    wolfe_config = Config()
    wolfe_config.wolfe_second_c = wolfe_c
    line_search = WolfeLineSearch(armijo_config)
    line_search.name = f"{line_search.name} (c={wolfe_c})"
    wolfe_line_searches.append(line_search)

In [ ]:
wolfe_optim_res = experiment_runner(
    a1a_oracle, wolfe_line_searches, grad_descent_optimizer, a1a_start_point
)

In [ ]:
wolfe_figure = experiment_graphic_builder(
    wolfe_optim_res,
    "Compare different parameters for Wolfe strong condition",
    a1a_true_minimum,
    smooth=True
)
wolfe_figure.show()

# Оптимизация методом Ньютона с использованием разложения Холецкого

In [ ]:
nc_optimizer = make_optimizer("newton-cholesky", config)

## a1a dataset

In [ ]:
%%time
a1a_optim_res = experiment_runner(a1a_oracle, line_searches[:-1], nc_optimizer, a1a_start_point)

In [ ]:
a1a_figure = experiment_graphic_builder(
    a1a_optim_res,
    "Compare line search methods on a1a dataset using Newton-Cholesky optimizer",
    a1a_true_minimum
)
a1a_figure.show()

## Breast cancer (scaled)

In [ ]:
%%time
bc_optim_res = experiment_runner(bc_oracle, line_searches[:-1], nc_optimizer, bc_start_point)

In [ ]:
bc_figure = experiment_graphic_builder(
    bc_optim_res,
    "Compare line search methods on breast cancer (scaled) dataset using Newton-Cholesky optimizer",
    bc_true_minimum
)
bc_figure.show()

## Random dataset

In [ ]:
%%time
random_optim_res = experiment_runner(random_oracle, line_searches[:-1], nc_optimizer, random_start_point)

In [ ]:
random_figure = experiment_graphic_builder(
    random_optim_res,
    "Compare line search methods on random dataset using Newton-Cholesky optimizer",
    random_true_minimum,
)
random_figure.show()

# Оптимизация методом Hessian-free Newton

In [ ]:
hfn_optimizer = make_optimizer("hessian free newton", config)

## a1a dataset

In [ ]:
%%time
a1a_optim_res = experiment_runner(a1a_oracle, line_searches[:-1], hfn_optimizer, a1a_start_point)

In [ ]:
a1a_figure = experiment_graphic_builder(
    a1a_optim_res,
    "Compare line search methods on a1a dataset using Newton-Cholesky optimizer",
    a1a_true_minimum
)
a1a_figure.show()

## Breast cancer (scaled)

In [ ]:
%%time
bc_optim_res = experiment_runner(bc_oracle, line_searches[:-1], hfn_optimizer, bc_start_point)

In [ ]:
bc_figure = experiment_graphic_builder(
    bc_optim_res,
    "Compare line search methods on breast cancer (scaled) dataset using Newton-Cholesky optimizer",
    bc_true_minimum
)
bc_figure.show()

## Random dataset

In [ ]:
%%time
random_optim_res = experiment_runner(random_oracle, line_searches[:-1], hfn_optimizer, random_start_point)

In [ ]:
random_figure = experiment_graphic_builder(
    random_optim_res,
    "Compare line search methods on random dataset using Newton-Cholesky optimizer",
    random_true_minimum,
)
random_figure.show()

# Сравнение методов между собой

In [ ]:
armijo_line_search = ArmijoLineSearch(config)
brent_line_search = BrentLineSearch(config)
across_optim_res = {
    "gradient descent": grad_descent_optimizer.optimize(a1a_oracle, armijo_line_search, a1a_start_point),
    "newton-cholesky": nc_optimizer.optimize(a1a_oracle, brent_line_search, a1a_start_point),
    "hessian-free newton": hfn_optimizer.optimize(a1a_oracle, brent_line_search, a1a_start_point)
}

In [ ]:
across_figure = experiment_graphic_builder(
    across_optim_res,
    "Compare line search methods on random dataset using Newton-Cholesky optimizer",
    a1a_true_minimum,
)
across_figure.show()